In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [39]:
import os
from utils import boxplots_utils
import pandas as pd
from termcolor import colored
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
from natsort import natsorted
import sys
import pickle
import seaborn as sns

# Display pandas df without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)



In [3]:
num_robots = 24
num_areas = 8

In [ ]:
results_folder = '/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/results_adaptive/random_mixed/random_mixed_2021-05-05_robots#24_timeout#6_redAreas#4_1800seconds'

In [ ]:
if not os.path.isdir(results_folder):
    print(colored("Error, " + results_folder + " does not exist", 'red'))
else:
    print(colored("OK, " + results_folder + " exists", 'green'))

In [ ]:
# df_area_client = pd.DataFrame()
# df_area_server = pd.DataFrame()
# df_task_client = pd.DataFrame()
# df_task_server = pd.DataFrame()
# df_kilo_client = pd.DataFrame()
# df_kilo_server = pd.DataFrame()

for filename in natsorted(os.listdir(results_folder)):
#     print('\t filename: ', filename)
    seed = filename.split("_")[0]
    if seed == 'seed#2':
        break
    
    if filename.endswith("areaLOG_client.tsv"):
        if not os.path.getsize(os.path.join(results_folder, filename)) > 0:
            print(colored("\tWARNING, empty file at:" + filename, 'red'))
            continue
        print('\tfilename: ', filename)
        df_area_client = pd.read_csv(os.path.join(results_folder, filename), sep="\t",
                                     header=None)

    if filename.endswith("areaLOG_server.tsv"):
        if not os.path.getsize(os.path.join(results_folder, filename)) > 0:
            print(colored("\tWARNING, empty file at:" + filename, 'red'))
            continue
        print('\tfilename: ', filename)
        df_area_server = pd.read_csv(os.path.join(results_folder, filename), sep="\t",
                                     header=None)

    if filename.endswith("taskLOG_client.tsv"):
        if not os.path.getsize(os.path.join(results_folder, filename)) > 0:
            print(colored("\tWARNING, empty file at:" + filename, 'red'))
            continue
        print('\tfilename: ', filename)
        df_task_client = pd.read_csv(os.path.join(results_folder, filename), sep="\t",
                                     header=None)

    if filename.endswith("taskLOG_server.tsv"):
        if not os.path.getsize(os.path.join(results_folder, filename)) > 0:
            print(colored("\tWARNING, empty file at:" + filename, 'red'))
            continue
        print('\tfilename: ', filename)
        df_task_server = pd.read_csv(os.path.join(results_folder, filename), sep="\t",
                                     header=None)

    if filename.endswith("kiloLOG_client.tsv"):
        if not os.path.getsize(os.path.join(results_folder, filename)) > 0:
            print(colored("\tWARNING, empty file at:" + filename, 'red'))
            continue
        print('\tfilename: ', filename)
        df_kilo_client = pd.read_csv(os.path.join(results_folder, filename), sep="\t",
                                     header=None)

    if filename.endswith("kiloLOG_server.tsv"):
        if not os.path.getsize(os.path.join(results_folder, filename)) > 0:
            print(colored("\tWARNING, empty file at:" + filename, 'red'))
            continue
        print('\tfilename: ', filename, end='\n')
        df_kilo_server = pd.read_csv(os.path.join(results_folder, filename), sep="\t",
                                     header=None)

In [ ]:
seed

### Kilo LOG part

In [ ]:
if len(df_kilo_client.columns) > 145:
    print("Cutting null elements in client kilo df")
    df_kilo_client.drop(df_kilo_client.columns[len(df_kilo_client.columns) - 1], axis=1, inplace=True)

if len(df_kilo_server.columns) > 145:
    print("Cutting null elements in server kilo df")
    df_kilo_server.drop(df_kilo_server.columns[len(df_kilo_server.columns) - 1], axis=1, inplace=True)

col_kilo_labels = ['time']
for i in range(0, len(df_kilo_server.columns) - 1, 6):
    #     print(i,end=", ")
    col_kilo_labels += ['id' + str(i // 6), 'state' + str(i // 6), 'posx' + str(i // 6), 'posy' + str(i // 6),
                        'ori' + str(i // 6), 'same_state' + str(i // 6)]

col_kilo_to_drop = []
for i in range((len(df_kilo_server.columns) - 1) // 6):
    #     print(i,end=", ")
    col_kilo_to_drop += ['same_state' + str(i)]

df_kilo_server.columns = col_kilo_labels
df_kilo_client.columns = col_kilo_labels
df_kilo_server = df_kilo_server.drop(col_kilo_to_drop, axis=1)
df_kilo_client = df_kilo_client.drop(col_kilo_to_drop, axis=1)

# kID0_client = df_kilo_client.iloc[:, :6]
# kID0_server = df_kilo_server.iloc[:, :6]

### area LOG part

In [ ]:
col_area_labels = ['time']
for i in range(0, len(df_area_server.columns) - 2, 6):
    # print(i, end=", ")
    col_area_labels += ['id' + str(i // 6), 'posx' + str(i // 6), 'posy' + str(i // 6), 'color' + str(i // 6),
                        'completed' + str(i // 6), 'contained' + str(i // 6)]

# Remove last empty col and assign labels to df_area_server
if len(df_area_server.columns) > 49:
    print("Cutting null elements in area server df")
    df_area_server.drop(df_area_server.columns[len(df_area_server.columns) - 1], axis=1, inplace=True)
df_area_server.columns = col_area_labels

# First df_area_client row contains garbage, so is substituted with the second row except for the time,
# then remove Nan values in [:,49:]
if len(df_area_client.columns) > 49:
    print("Cutting null elements in area client df")
    df_area_client.loc[0, 1:] = df_area_client.loc[1, 1:]
    df_area_client = df_area_client.drop(np.arange(49, len(df_area_client.columns)), axis=1)
df_area_client.columns = col_area_labels

### completed task LOG part

In [ ]:
# task_label = ['time', 'id', 'creationTime', 'completitionTime', 'color', 'contained']
# df_task_client.columns = task_label

#### Get area positions and colors

In [ ]:
area_color_label = []
for i in range(num_areas):
    area_color_label += ["color"+str(i)]
#     print("color"+str(i))
# area_color_label

areas_client_color = df_area_client[area_color_label].iloc[0, :].values
areas_server_color = df_area_server[area_color_label].iloc[0, :].values

print(areas_client_color)
print(areas_server_color)

In [ ]:
area_pos_label = []
for i in range(num_areas):
    area_pos_label += ["posx"+str(i)]
    area_pos_label += ["posy"+str(i)]
# area_pos_label

areas_pos = df_area_client[area_pos_label].iloc[0, :].values

print(areas_pos)

In [ ]:
areas_pos = areas_pos.reshape(-1,2)
areas_pos

In [ ]:
df_kilo_client_temp = df_kilo_client.copy()#.iloc[:20,:26]
df_kilo_server_temp = df_kilo_server.copy()
df_kilo_server_temp.iloc[:10,:]

In [ ]:
# for kilo_id in np.arange(1, len(df_kilo_client_temp.columns), 5):
#     print(colored("kilo_id:"+str((kilo_id-1)//5), 'blue'))
# #     print(df_kilo_client.iloc[:20, kilo_id+2:kilo_id+4].values, end='\n\n')
#     kilo_pos = df_kilo_client_temp.iloc[:5, kilo_id+2:kilo_id+4].values
#     in_area=np.empty(kilo_pos.shape[0], dtype=int); in_area.fill(-1)
#     for area_idx, area_pos in enumerate(areas_pos):
#     #     print(area_idx,' ', area_pos)
#         dist = np.linalg.norm(kilo_pos-area_pos, axis=1)
#     #     print(dist, end='\n\n')
#         in_area = np.where(dist < 0.08, df_area_colors.iloc[area_idx][-1], in_area)
# #     in_area = np.where(in_area == -1, np.NaN, in_area)
#     print(in_area)
    

## Area color df

In [ ]:
color_list = ["color" + str(i) for i in range(num_areas)]

In [ ]:
df_area3_s = df_area_server.iloc[:1, :][color_list]
df_area3_c = df_area_client.iloc[:1, :][color_list]
# [['color0', 'color1', 'color2', 'color3', 'color4', 'color5', 'color6', 'color7']]
#[['color0', 'completed0', 'color1', 'completed1', 'color2', 'completed2', 'color3', 'completed3', 'color4', 'completed4', 'color5', 'completed5', 'color6', 'completed6', 'color7', 'completed7']]

In [ ]:
for i,idx in enumerate(range(1,len(df_area3_c.columns)*2,2)):
#     print(i, ' ', idx)
    df_area3_c.insert(loc=idx, column='other_col'+str(i), value=df_area3_s.iloc[0][i])
    
df_area3_c

In [ ]:
client = [col for col in df_area3_c.columns if 'color' in col]
server = [col for col in df_area3_c.columns if 'other_col' in col]
df_area_colors = pd.lreshape(df_area3_c, {'color_client' : client, 'color_server' : server})
df_area_colors

In [ ]:
area_type = []
for area in df_area_colors.values:
    if(area[0] == 0 and area[1] == 0):
        area_type += ['BB']
    if(area[0] == 0 and area[1] == 1):
        area_type += ['BR']
    if(area[0] == 1 and area[1] == 0):
        area_type += ['RB']
    if(area[0] == 1 and area[1] == 1):
        area_type += ['RR']
area_type

In [ ]:
df_area_colors.insert(loc=2, column='area_type', value=area_type)

In [ ]:
df_area_colors

## Per il server:

In [ ]:
for i,kilo_id in enumerate(np.arange(1, len(df_kilo_server_temp.columns), 5)):
    print(colored("kilo_id:"+str((kilo_id-1)//5), 'blue'))
#     print(df_kilo_client_temp.iloc[:20, kilo_id+2:kilo_id+4].values, end='\n\n')
    kilo_pos = df_kilo_server_temp.iloc[:, kilo_id+i+2:kilo_id+i+4].values
#     print(kilo_pos)
    in_area=np.empty(kilo_pos.shape[0], dtype=int); in_area.fill(-1)
    for area_idx, area_pos in enumerate(areas_pos):
    #     print(area_idx,' ', area_pos)
        dist = np.linalg.norm(kilo_pos-area_pos, axis=1)
    #     print(dist, end='\n\n')
        in_area = np.where(dist < 0.1, df_area_colors.iloc[area_idx][-1][::-1], in_area)
#     in_area = np.where(in_area == -1, np.NaN, in_area)
#     print(in_area)
    df_kilo_server_temp.insert(loc=int(kilo_id+i+2), column='area_type'+str(i), value=in_area)

In [ ]:
df_kilo_server_temp.iloc[:2,:]

## Per il client:

In [ ]:
for i,kilo_id in enumerate(np.arange(1, len(df_kilo_client_temp.columns), 5)):
    print(colored("kilo_id:"+str((kilo_id-1)//5), 'blue'))
#     print(df_kilo_client_temp.iloc[:20, kilo_id+2:kilo_id+4].values, end='\n\n')
    kilo_pos = df_kilo_client_temp.iloc[:, kilo_id+i+2:kilo_id+i+4].values
#     print(kilo_pos)
    in_area=np.empty(kilo_pos.shape[0], dtype=int); in_area.fill(-1)
    for area_idx, area_pos in enumerate(areas_pos):
    #     print(area_idx,' ', area_pos)
        dist = np.linalg.norm(kilo_pos-area_pos, axis=1)
    #     print(dist, end='\n\n')
        in_area = np.where(dist < 0.1, df_area_colors.iloc[area_idx][-1], in_area)
#     in_area = np.where(in_area == -1, np.NaN, in_area)
#     print(in_area)
    df_kilo_client_temp.insert(loc=int(kilo_id+i+2), column='area_type'+str(i), value=in_area)

In [ ]:
df_kilo_client_temp.iloc[:2,:]

## Jump to Estrazione di stato...

#### Save & Load pickle

In [ ]:
# df_kilo_client_temp.to_pickle('porcodddio')
# df_kilo_client_temp1= pd.read_pickle('porcodddio')
# df_kilo_client_temp1

In [ ]:
# Test

# in_area=np.empty(kID0_client_pos.shape[0], dtype=int); in_area.fill(-1)
# # print(in_area)
# for area_idx, area_pos in enumerate(areas_pos):
# #     print(area_idx,' ', area_pos)
#     dist = np.linalg.norm(kID0_client_pos-area_pos, axis=1)
# #     print(dist, end='\n\n')
#     in_area = np.where(dist < 0.08, df_area_colors.iloc[area_idx][-1], in_area)
# # in_area = np.where(in_area == -1, np.NaN, in_area)

In [ ]:
# in_area

In [ ]:
df_kilo_client_temp = df_kilo_client.iloc[:20, :6]
df_kilo_client_temp
df_kilo_client_temp.insert(loc=5, column='area_type', value=in_area)
df_kilo_client_temp

In [ ]:
# completed = df_area_client.iloc[:20, :][['completed1']]
# contained = df_area_client.iloc[:20, :][['contained1']]

In [ ]:
# df_kilo_client_temp.insert(loc=6, column='completed1' , value=completed)
# df_kilo_client_temp.insert(loc=7, column='contained1' , value=contained)

In [ ]:
# df_kilo_client_temp

In [ ]:
for idx, elem in enumerate(in_area):
#     print(idx, ' ', elem)
#     print(elem)
    if(elem != -1):
        print(df_area_colors.iloc[elem][-1])
#         print("hello")

In [ ]:
df_area_colors.iloc[1].values

In [ ]:
# Evaluate transition 0->1, 1->0
# df_transition = kID0_client[['state0']].rolling(2).apply(lambda x: x[0] != x[-1], raw=True).sum().astype(int)
# df_transition

## Estrazione di stato e tipo di area per ogni kilobot

In [ ]:
kilo_resume = [["state" + str(i), "area_type" + str(i)] for i in range(num_robots)]

In [ ]:
kilo_resume = np.reshape(kilo_resume, (-1))

In [ ]:
print(kilo_resume)

In [ ]:
server_kilo_resume = df_kilo_server_temp.iloc[:][kilo_resume]
client_kilo_resume = df_kilo_client_temp.iloc[:][kilo_resume]

In [ ]:
client_kilo_resume.iloc[:2,:]

In [ ]:
server_kilo_resume.iloc[:2,:]

In [ ]:
total_exp_df = client_kilo_resume.join(server_kilo_resume, lsuffix='_c', rsuffix='_s')

In [ ]:
total_exp_df.iloc[:,4:6]

In [ ]:
# small_total_exp_df = total_exp_df.iloc[:,0:2]

In [ ]:
# small_total_exp_df = small_total_exp_df.replace(2, 3)

In [ ]:
# small_total_exp_df.iloc[:1,:]

In [ ]:
# mask = small_total_exp_df["state4_c"].shift().ne(small_total_exp_df["state4_c"])

In [ ]:
# mask

In [ ]:
# small_client_kilo = client_kilo_resume.iloc[:,0:2]
# small_client_kilo = small_client_kilo.replace(2,3)
# small_client_kilo

In [ ]:
# mask1 = (small_client_kilo[small_client_kilo.columns.values[0]].diff() == 2)#.sum(axis=0)

In [ ]:
# small_client_kilo[mask1]

In [ ]:
# small_client_kilo[mask1][small_client_kilo.columns.values[1]].value_counts().to_frame().T

In [ ]:
# len(server_kilo_resume.columns)

##### For each kilobot evaluate completed area type

In [ ]:
completed_area_count = pd.DataFrame(columns=['RR', 'RB', 'BR', 'BB'])
for i in range(0,len(total_exp_df.columns),2):
#     print(total_exp_df.iloc[:50,i:i+2])
    print('\n',i)
    kilo_state = total_exp_df.iloc[:,i:i+2]
#     kilo_state = kilo_state.replace(2,3)
    mask = (kilo_state[kilo_state.columns.values[0]].diff() == -1)
    print(kilo_state[mask])
    print(kilo_state[mask][kilo_state.columns.values[1]].value_counts(), end='\n\n')
    robot_timeout = kilo_state[mask][kilo_state.columns.values[1]].value_counts().to_frame().T
#     robot_timeout = pd.DataFrame(kilo_state[mask][kilo_state.columns.values[1]].value_counts(), columns=['RR, RB,BR,BB'])
#     print(robot_timeout)
    completed_area_count = completed_area_count.append(robot_timeout)
    print(robot_timeout, end='\n\n')
    
completed_area_count = completed_area_count.fillna(0)

In [ ]:
completed_area_count

##### For each kilobot evaluate elapsed timeout

In [ ]:
timeout_count = pd.DataFrame(columns=['RR', 'RB', 'BR', 'BB'])
for i in range(0,len(total_exp_df.columns),2):
#     print(total_exp_df.iloc[:50,i:i+2])
    print('\n',i)
    kilo_state = total_exp_df.iloc[:,i:i+2]
    kilo_state = kilo_state.replace(2,3)
    mask = (kilo_state[kilo_state.columns.values[0]].diff() == 2)
#     print(kilo_state[mask])
#     print(kilo_state[mask][kilo_state.columns.values[1]].value_counts(), end='\n\n')
    robot_timeout = kilo_state[mask][kilo_state.columns.values[1]].value_counts().to_frame().T
#     robot_timeout = pd.DataFrame(kilo_state[mask][kilo_state.columns.values[1]].value_counts(), columns=['RR, RB,BR,BB'])
#     print(robot_timeout)
    timeout_count = timeout_count.append(robot_timeout)
    print(robot_timeout, end='\n\n')
    
timeout_count = timeout_count.fillna(0)

In [ ]:
timeout_count

In [ ]:
# df.mean non considera i NaN values
single_run_mean = timeout_count.mean(axis = 0)

In [ ]:
single_run_mean

In [ ]:
seed

In [ ]:
single_df = single_run_mean.to_frame().T
single_df.index = [seed]
single_df

In [ ]:
# single_run_mean.to_frame().T.to_pickle(os.path.join(results_folder,seed+"_mean_elapsed_timeout.pickle"))

In [ ]:
run_memory_mean = pd.DataFrame()
if os.path.isfile(os.path.join(results_folder,"mean_elapsed_timeout.pickle")):
    print("Appending")
    run_memory_mean = pd.read_pickle(os.path.join(results_folder,"mean_elapsed_timeout.pickle"))
    run_memory_mean = run_memory_mean.append(single_df)
    run_memory_mean.to_pickle(os.path.join(results_folder,"mean_elapsed_timeout.pickle"))
else:
    print("Writing")
    single_df.to_pickle(os.path.join(results_folder,"mean_elapsed_timeout.pickle"))

In [ ]:
run_memory_mean

In [ ]:
df_prova = pd.read_pickle(os.path.join(results_folder,"mean_elapsed_timeout.pickle"))
df_prova

In [ ]:
results_folder1 = '/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/results_adaptive/random_mixed'
df_prova = pd.read_pickle(os.path.join(results_folder1,"mean_elapsed_timeout10.pickle"))

In [ ]:
df_prova

In [ ]:
for filename in natsorted(os.listdir(results_folder1)):
    if filename.startswith("mean_elapsed"):
        print('filename: ', filename)
        df_temp = pd.read_pickle(os.path.join(results_folder1,filename))
        print(df_temp)

### Plot the timeout boxplot

In [ ]:
def set_box_color(bp, color):
    """
    # Set boxplot rules
    """
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)


def grouped_4_boxplot(rr, rb, br, bb, fig_name):
    """
    # Boxplot from three dicts (red,blue,mixed)
    :param red: dict with red area values
    :param blue: dict with blue area values
    :param mixed: dict with red-blue area values
    :param fig_name: name of the output figure, saved in the current directory
    :return:
    """
    ticks = rr.keys()

    fig, ax = plt.subplots(figsize=(10, 5), dpi=80)

    bpRR = plt.boxplot(rr.values(), positions=np.array(range(len(rr.values())))*3.0-0.4, sym='', widths=0.3)
    bpRB = plt.boxplot(rb.values(), positions=np.array(range(len(rb.values())))*3.0+0.4, sym='', widths=0.3)
    bpBR = plt.boxplot(br.values(), positions=np.array(range(len(br.values())))*3.0, sym='', widths=0.3)
#     bpBB = plt.boxplot(bb.values(), positions=np.array(range(len(bb.values())))*3.0, sym='', widths=0.3)

    set_box_color(bpRR, '#d95f02') # colors are from http://colorbrewer2.org/
    set_box_color(bpRB, '#ffd700')
    set_box_color(bpBR, '#ffA500')
#     set_box_color(bpBB, '#7570b3')

    # draw temporary rr and blue lines and use them to create a legend
    plt.plot([], c='#d95f02', label='RR')
    plt.plot([], c='#ffd700', label='RB')
    plt.plot([], c='#ffA500', label='BR')
#     plt.plot([], c='#7570b3', label='BB')
    plt.legend(loc=2)

    # plt.xticks(range(0, 300, 5), ticks)
    # plt.xlim(-3, len(ticks)*3)
    plt.ylim(-1, 44)
    plt.xticks(range(0, len(ticks) * 3, 3), ticks)
    plt.xlim(-2, len(ticks)*3)

    ax.set_xticklabels(rr.keys())
    ax.set_xlabel("Timeout[s]")
    ax.set_ylabel("Completed areas")

    plt.tight_layout()
#     plt.savefig(fig_name + '.png')

In [ ]:
results_folder1 = '/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/results_adaptive/random_mixed'
df_prova = pd.read_pickle(os.path.join(results_folder1,"mean_run_elapsed_timeout120.pickle"))

In [ ]:
# df_prova

In [ ]:
# df_prova.to_dict()

In [ ]:
# df_prova.to_dict()['RR']

In [ ]:
prova_dict = dict(df_prova['RR'])

In [ ]:
type(df_prova.to_dict('series')['RR'])

In [ ]:
# df_prova.to_dict('series')['RR']
# for k, val in df_prova.to_dict('series')['RR'].items():
#     print("type k:", type(k))
#     print("type val:", type(val))
#     print("k:", k, " val:",val)

In [ ]:
# df_prova['RR'].values

In [ ]:
# print(df_prova.to_dict('r'))

In [ ]:
# print(df_prova.to_dict())

In [ ]:
rr.shape[1]

In [ ]:
rr.values

In [ ]:
rr.shape

In [ ]:
df_prova

In [ ]:
# rr = df_prova['RR']
# rb = df_prova['RB'] 
# br = df_prova['BR']
# bb = df_prova['BB']

In [ ]:
len(rr.values())

In [ ]:
def grouped_4_boxplot(rr, rb, br, bb, fig_name):
    ticks = rr.keys()

    fig, ax = plt.subplots(figsize=(20, 5), dpi=80)

    bpRR = plt.boxplot(rr.values(), positions=np.array(range(len(rr.values())))*3.0-0.8, sym='', widths=0.35)
    bpRB = plt.boxplot(rb.values(), positions=np.array(range(len(rb.values())))*3.0-0.25, sym='', widths=0.35)
    bpBR = plt.boxplot(br.values(), positions=np.array(range(len(rr.values())))*3.0+0.25, sym='', widths=0.35)
    bpBB = plt.boxplot(bb.values(), positions=np.array(range(len(bb.values())))*3.0+0.8, sym='', widths=0.35)

    set_box_color(bpRR, '#d95f02') # colors are from http://colorbrewer2.org/
    set_box_color(bpRB, '#1c9099')
    set_box_color(bpBR, '#2ca25f')
    set_box_color(bpBB, '#7570b3')

    # draw temporary rr and blue lines and use them to create a legend
    plt.plot([], c='#d95f02', label='RR')
    plt.plot([], c='#1c9099', label='RB')
    plt.plot([], c='#2ca25f', label='BR')
    plt.plot([], c='#7570b3', label='BB')
    plt.legend(loc=2)

    # plt.xticks(range(0, 300, 5), ticks)
    # plt.xlim(-3, len(ticks)*3)
    plt.ylim(-1, 15)
    plt.xticks(range(0, len(ticks) * 3, 3), ticks)
    plt.xlim(-2, len(ticks)*3)

    ax.set_xticklabels(rr.keys())
    ax.set_xlabel("Timeout[s]")
    ax.set_ylabel("Elapsed timeout for each kilobot")

    plt.tight_layout()
    plt.savefig(fig_name + '.png')

In [ ]:
results_folder = os.path.join(os.getcwd(), "results_adaptive/" + experiment)
if not os.path.isdir(results_folder):
    print(colored("Error, " + results_folder + " does not exist", 'red'))
else:
    print(colored("OK, " + results_folder + " exists", 'green'))

rr = {}
rb = {}
br = {}
bb = {}

for mean_timeout in natsorted(os.listdir(os.path.join(results_folder))):
    if not mean_timeout.startswith("mean_run"):
        continue
    
    print(colored("Timeout file:", 'blue'), mean_timeout)
    
    parameters = mean_timeout.split("_")
    for param in parameters:
        if param.startswith("timeout"):
            timeout = int(param.split("#")[-1])
    
    df_timeout = pd.read_pickle(os.path.join(results_folder,mean_timeout))
    rr[timeout] = df_timeout['RR'].values
    rb[timeout] = df_timeout['RB'].values
    br[timeout] = df_timeout['BR'].values
    bb[timeout] = df_timeout['BB'].values    
#     if mean_timeout.endswith("10.pickle"):
#         break

In [ ]:
figname = 'meanElapsedTimeout_groupsize0.75_randomMixed_adaptive.png'
grouped_4_boxplot(rr,rb,br,bb,figname)

In [ ]:
df_timeout = pd.read_pickle(os.path.join(results_folder,mean_timeout))
rr[mean_timeout[-9:-7]] = df_timeout['RR'].values

In [ ]:
rr

In [ ]:
some_array = df_timeout['RR'].values
some_array = some_array.reshape(-1)

In [ ]:
some_array.shape

# Position distribution

In [ ]:
robot_pos_label = [["posx" + str(i), "posy" + str(i)] for i in range(num_robots)]
robot_pos_label = np.reshape(robot_pos_label, (-1))
robot_pos_label = np.append('time', robot_pos_label)
robot_pos_label

In [ ]:
kilo_0_pos = df_kilo_client_temp[robot_pos_label].iloc[:,3:5]
kilo_0_pos

In [ ]:
dist = np.linalg.norm(kilo_0_pos, axis=1)
dist

In [ ]:
robot_origin_distance_df = pd.DataFrame()
for r_idx in range(1,len(robot_pos_label),2):
#     print(df_kilo_client_temp[robot_pos_label].iloc[:2,r_idx:r_idx+2])
    print(r_idx//2)
    r_pos = df_kilo_client_temp[robot_pos_label].iloc[:,r_idx:r_idx+2]
    origin_dist = np.linalg.norm(r_pos, axis=1)
    robot_origin_distance_df['dist'+str(r_idx//2)] = origin_dist

In [ ]:
len(robot_pos_label)

In [ ]:
robot_origin_distance_df['distance0'] = [44]

In [ ]:
robot_origin_distance_df.index = df_kilo_client_temp['time']
robot_origin_distance_df.shape

In [ ]:
bin_edges = np.linspace(0, 0.5, 20)
bin_edges

In [ ]:
hist_val = np.array([])
i=0
for x in robot_origin_distance_df.values:
#     print(x)
    hist, _ = np.histogram(x, bin_edges)
#     print(i,' ',hist)
    i+=1
    hist_val = np.vstack([hist_val, hist]) if hist_val.size else hist
    print(i, '\n', hist_val, end='\n\n')

# for i in range(edges[1:].size):
#     area = np.pi * (np.square(edges[1:][i]) - np.square(edges[1:][i - 1])) if i else np.pi * np.square(edges[1:][i])
#     hist_val[:, i] = np.true_divide(hist_val[:, i], area * runs)

In [ ]:
hist_val.shape

In [ ]:
values = hist_val.T
y_edges = bin_edges[1:].round(decimals=3)
fig = plt.figure(figsize=(10, 5), dpi=80)
# plt.ylabel('Distance from origin')
# plt.xlabel('time(s)')
# plt.legend()
yticks = y_edges
# plt.imshow(distances,interpolation='none')



# print("\t v_max:", v_max)
ax = sns.heatmap(values, yticklabels=yticks, cmap="viridis", vmin=0)#, vmax=v_max)
ax.set_title("Robots diffusion")
ax.set_ylabel('distance from the origin')
ax.set_xlabel('time')
plt.savefig("randomMixed_adaptive_timeout6")

# Prova di impilazione df

In [ ]:
df_kilo_client_temp.shape

In [ ]:
df_kilo_server_temp.shape

In [ ]:
df_kilo_client_temp.iloc[898:,]

In [ ]:
df_kilo_server_temp.iloc[898:,]

In [ ]:
df_join = pd.DataFrame()
df_join = df_kilo_client_temp.append(df_kilo_server_temp)

In [ ]:
df_join

In [11]:
num_robots = 24
num_areas = 8

In [12]:
results_folder = '/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/results_adaptive/random_mixed/'
if not os.path.isdir(results_folder):
    print(colored("Error, " + results_folder + " does not exist", 'red'))
else:
    print(colored("OK, " + results_folder + " exists", 'green'))

OK, /home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/results_adaptive/random_mixed/ exists


In [83]:
df_kilo_timeout = pd.DataFrame()

evaluate_run = False

for timeout_folder in natsorted(os.listdir(os.path.join(results_folder))):

    if timeout_folder.endswith("pickle"):
        continue

    print(colored("Timeout folder:", 'blue'), timeout_folder)
    timeout = -1
    parameters = timeout_folder.split("_")
    for param in parameters:
        if param.startswith("timeout"):
            timeout = int(param.split("#")[-1]) * 10
            # print("\t timeoutR:",timeoutR)

    if timeout == -1:
        print(colored("\tWARNING: wrong timeout folder", 'red'))
        continue

    # TODO : check for total_timeout_df existence
    if os.path.isfile(os.path.join(results_folder, timeout_folder, "kiloLOG_timeout#"+str(timeout)+"_.pickle")):
        print("Already exists ", os.path.join(results_folder, timeout_folder, "kiloLOG_timeout#"+str(timeout)+"_.pickle"))
    else:
        # print(colored(
        #     os.path.join(results_folder, pickle_file_root"_timeout#"+str(timeout*10)+"_.pickle"),
        #     'red'))
        # sys.exit()
        for filename in natsorted(os.listdir(os.path.join(results_folder, timeout_folder))):
            filename_seed = filename.split("_")[0].split("#")[-1]
            
#             if filename_seed == '3':
#                 filename_seed = '2'
#                 break
            
            
            # print(filename)
            if filename.endswith("areaLOG_client.tsv"):
                if not os.path.getsize(os.path.join(results_folder, timeout_folder, filename)) > 0:
                    print(colored("\tWARNING, empty file at:" + filename, 'red'))
                    continue
                print('\tfilename: ', filename)
                
                df_area_client = pd.read_csv(os.path.join(results_folder, timeout_folder, filename), sep="\t", header=None)

            if filename.endswith("areaLOG_server.tsv"):
                if not os.path.getsize(os.path.join(results_folder, timeout_folder, filename)) > 0:
                    print(colored("\tWARNING, empty file at:" + filename, 'red'))
                    continue
                print('\tfilename: ', filename)
                df_area_server = pd.read_csv(os.path.join(results_folder, timeout_folder, filename), sep="\t", header=None)

            if filename.endswith("kiloLOG_client.tsv"):
                if not os.path.getsize(os.path.join(results_folder, timeout_folder, filename)) > 0:
                    print(colored("\tWARNING, empty file at:" + filename, 'red'))
                    continue
                print('\tfilename: ', filename)
                df_kilo_client = pd.read_csv(os.path.join(results_folder, timeout_folder, filename), sep="\t", header=None)

            if filename.endswith("kiloLOG_server.tsv"):
                if not os.path.getsize(os.path.join(results_folder, timeout_folder, filename)) > 0:
                    print(colored("\tWARNING, empty file at:" + filename, 'red'))
                    continue
                print('\tfilename: ', filename, end='\n')
                df_kilo_server = pd.read_csv(os.path.join(results_folder, timeout_folder, filename), sep="\t",header=None)
                evaluate_run = True
                
                
            if evaluate_run:
                print(colored("\tEvaluating run:"+filename_seed, 'blue'))
                
                if len(df_kilo_client.columns) > 145:
                    print("\tCutting null elements in client kilo df")
                    df_kilo_client.drop(df_kilo_client.columns[len(df_kilo_client.columns) - 1], axis=1, inplace=True)

                if len(df_kilo_server.columns) > 145:
                    print("\tCutting null elements in server kilo df")
                    df_kilo_server.drop(df_kilo_server.columns[len(df_kilo_server.columns) - 1], axis=1, inplace=True)

                col_kilo_labels = ['time']
                for i in range(0, len(df_kilo_server.columns) - 1, 6):
                    #     print(i,end=", ")
                    col_kilo_labels += ['id' + str(i // 6), 'state' + str(i // 6), 'posx' + str(i // 6), 'posy' + str(i // 6),
                                        'ori' + str(i // 6), 'same_state' + str(i // 6)]

                col_kilo_to_drop = []
                for i in range((len(df_kilo_server.columns) - 1) // 6):
                    #     print(i,end=", ")
                    col_kilo_to_drop += ['same_state' + str(i)]

                df_kilo_server.columns = col_kilo_labels
                df_kilo_client.columns = col_kilo_labels
                df_kilo_server = df_kilo_server.drop(col_kilo_to_drop, axis=1)
                df_kilo_client = df_kilo_client.drop(col_kilo_to_drop, axis=1)

                col_area_labels = ['time']
                for i in range(0, len(df_area_server.columns) - 2, 6):
                    # print(i, end=", ")
                    col_area_labels += ['id' + str(i // 6), 'posx' + str(i // 6), 'posy' + str(i // 6), 'color' + str(i // 6),
                                        'completed' + str(i // 6), 'contained' + str(i // 6)]

                # Remove last empty col and assign labels to df_area_server
                if len(df_area_server.columns) > 49:
                    print("\tCutting null elements in area server df")
                    df_area_server.drop(df_area_server.columns[len(df_area_server.columns) - 1], axis=1, inplace=True)
                df_area_server.columns = col_area_labels

                # First df_area_client row contains garbage, so is substituted with the second row except for the time,
                # then remove Nan values in [:,49:]
                if len(df_area_client.columns) > 49:
                    print("\tCutting null elements in area client df")
                    df_area_client.loc[0, 1:] = df_area_client.loc[1, 1:]
                    df_area_client = df_area_client.drop(np.arange(49, len(df_area_client.columns)), axis=1)
                df_area_client.columns = col_area_labels
                
                area_pos_label = []
                for i in range(num_areas):
                    area_pos_label += ["posx"+str(i)]
                    area_pos_label += ["posy"+str(i)]
                # area_pos_label

                areas_pos = df_area_client[area_pos_label].iloc[0, :].values
                areas_pos = areas_pos.reshape(-1,2)
#                 print(areas_pos)
                
                color_list = ["color" + str(i) for i in range(num_areas)]
                df_area3_s = df_area_server.iloc[:1, :][color_list]
                df_area3_c = df_area_client.iloc[:1, :][color_list]

                for i,idx in enumerate(range(1,len(df_area3_c.columns)*2,2)):
                #     print(i, ' ', idx)
                    df_area3_c.insert(loc=idx, column='other_col'+str(i), value=df_area3_s.iloc[0][i])

                df_area3_c
                client = [col for col in df_area3_c.columns if 'color' in col]
                server = [col for col in df_area3_c.columns if 'other_col' in col]
                df_area_colors = pd.lreshape(df_area3_c, {'color_client' : client, 'color_server' : server})
                area_type = []
                for area in df_area_colors.values:
                    if(area[0] == 0 and area[1] == 0):
                        area_type += ['BB']
                    if(area[0] == 0 and area[1] == 1):
                        area_type += ['BR']
                    if(area[0] == 1 and area[1] == 0):
                        area_type += ['RB']
                    if(area[0] == 1 and area[1] == 1):
                        area_type += ['RR']
                df_area_colors.insert(loc=2, column='area_type', value=area_type)
                
                for i,kilo_id in enumerate(np.arange(1, len(df_kilo_server.columns), 5)):
                    # print(colored("kilo_id:"+str((kilo_id-1)//5), 'blue'))
                #     print(df_kilo_client.iloc[:20, kilo_id+2:kilo_id+4].values, end='\n\n')
                    kilo_pos = df_kilo_server.iloc[:, kilo_id+i+2:kilo_id+i+4].values
                #     print(kilo_pos)
                    in_area=np.empty(kilo_pos.shape[0], dtype=int); in_area.fill(-1)
                    for area_idx, area_pos in enumerate(areas_pos):
                    #     print(area_idx,' ', area_pos)
                        dist = np.linalg.norm(kilo_pos-area_pos, axis=1)
                    #     print(dist, end='\n\n')
                        in_area = np.where(dist < 0.1, df_area_colors.iloc[area_idx][-1][::-1], in_area)
                #     in_area = np.where(in_area == -1, np.NaN, in_area)
                #     print(in_area)
                    df_kilo_server.insert(loc=int(kilo_id+i+2), column='area_type'+str(i), value=in_area)
                    
                for i,kilo_id in enumerate(np.arange(1, len(df_kilo_client.columns), 5)):
#                     print(colored("kilo_id:"+str((kilo_id-1)//5), 'blue'))
                #     print(df_kilo_client.iloc[:20, kilo_id+2:kilo_id+4].values, end='\n\n')
                    kilo_pos = df_kilo_client.iloc[:, kilo_id+i+2:kilo_id+i+4].values
                #     print(kilo_pos)
                    in_area=np.empty(kilo_pos.shape[0], dtype=int); in_area.fill(-1)
                    for area_idx, area_pos in enumerate(areas_pos):
                    #     print(area_idx,' ', area_pos)
                        dist = np.linalg.norm(kilo_pos-area_pos, axis=1)
                    #     print(dist, end='\n\n')
                        in_area = np.where(dist < 0.1, df_area_colors.iloc[area_idx][-1], in_area)
                #     in_area = np.where(in_area == -1, np.NaN, in_area)
                #     print(in_area)
                    df_kilo_client.insert(loc=int(kilo_id+i+2), column='area_type'+str(i), value=in_area)
                    
                df_kilo_single_run = df_kilo_client.join(df_kilo_server, lsuffix='_c', rsuffix='_s')
                df_kilo_single_run = df_kilo_single_run.set_index(df_kilo_single_run.index.astype(str) + '_' + filename_seed)
                
                df_kilo_timeout = df_kilo_timeout.append(df_kilo_single_run)
            
                
                evaluate_run = False
        df_kilo_timeout.to_pickle(os.path.join(results_folder, timeout_folder, "kiloLOG_timeout#"+str(timeout)+"_.pickle"))
        print("Saving at: ", os.path.join(results_folder, timeout_folder, "kiloLOG_timeout#"+str(timeout)+"_.pickle"))
        print("Changing dir")
    
    if(timeout == 20):
        break

Timeout folder: random_mixed_2021-05-05_robots#24_timeout#1_redAreas#4_1800seconds
Already exists  /home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/results_adaptive/random_mixed/random_mixed_2021-05-05_robots#24_timeout#1_redAreas#4_1800seconds/kiloLOG_timeout#10_.pickle
Timeout folder: random_mixed_2021-05-05_robots#24_timeout#2_redAreas#4_1800seconds
Already exists  /home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/results_adaptive/random_mixed/random_mixed_2021-05-05_robots#24_timeout#2_redAreas#4_1800seconds/kiloLOG_timeout#20_.pickle


In [84]:
df_kilo_timeout.shape

(0, 0)

In [90]:
dfPath = "/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/results_adaptive/random_mixed/random_mixed_2021-05-05_robots#24_timeout#1_redAreas#4_1800seconds/kiloLOG_timeout#10_.pickle"

In [91]:
df_timeout_20 = pd.read_pickle(dfPath)

In [92]:
df_timeout_20.shape

(89778, 290)

In [87]:
df_timeout_10.shape

(0, 0)

In [70]:
df_timeout_10.iloc[3000:3002,:2]

,time_c,id0_c
308_4,616.0,0
309_4,618.0,0
